In [ ]:
%matplotlib inline
import pandas as pd

import numpy as np

import matplotlib.pyplot as plt

from matplotlib.ticker import MaxNLocator

from IPython.display import display

import re

In [ ]:
col_names  = ["Street",  "Street_batch", "Street_cluster"]
filename = "blog_streets.csv.gz"


# filename = "Pays.xlsx"
# col_names = ["Pays", "Pays_clean"] 


# Functions

## Val_count

In [ ]:
def get_val_count(filename, col_name):
    if ".csv" in filename: df = pd.read_csv(filename, usecols = [col_name])
    elif ".xlsx" in filename: 
        df = pd.read_excel(filename)
        df = df[[col_name]]
        
    else: df = pd.read_table(filename, usecols = [col_name])
    print(col_name, ":")
    
    print(" - Number of records: {} ({} non empty)".format(df.shape[0], df[df[col_name].fillna("").astype(str).str.len()>0].shape[0]))
    
    print(" - Number of (distinct) values: ", df[col_name].nunique())
    print(" - Number of empty values: ", df[df[col_name].fillna("").str.len()==0].shape[0])
    
    
    
    val_counts = df[col_name].value_counts().reset_index(drop=True).reset_index()
    val_counts["index"] = val_counts["index"]+1
    val_counts = val_counts.reset_index(drop = True)

    val_counts["cum_sum"] = val_counts[col_name].cumsum()
    
    first_one = val_counts[val_counts[col_name] == 1].first_valid_index()
    if first_one: 
        print(" - Number of 1's: ", val_counts["index"].max()-first_one +1)

    val_counts = val_counts.rename(columns={"index":"ValueId", "cum_sum":"#CumRecords", col_name:"#Records"}).set_index("ValueId")
    val_counts.name = col_name
    return val_counts
    

In [ ]:
def plot_val_count(val_counts, ax=None, color="C0", rot=-45, scale_val= False, figsize=(15,10), stats = [10, 100, "med_y", "med_x", "p_90","t_1", "t_2", "max"], drawstyle="steps-post"):

    
    def add_lines(ax, val_counts, x, title):
        global all_xy_labels
        
        line_color = color 
        y = val_counts.loc[x]["#CumRecords"]
        

        ax.plot([x, x], [0, y], color=line_color, linestyle=':', marker='x', markevery=(1, 2), linewidth=1)
        ax.plot([0, x], [y, y], color=line_color, linestyle=':', linewidth=1)
        
        
        ax.text(x- max_x * 0.02 , y - np.sign(rot)*max_y * 0.02, "{} ({:.2f}%)".format(int(y), (y/max_y)*100), 
                color=color, ha='right', va="bottom" if rot<0 else "top", bbox=dict(facecolor='white', alpha=0.9,  edgecolor='none'), rotation=rot)
        
        ax.text(x, - max_y * 0.1, "{}: {} ({:.2f}%)".format(title, int(x), (x/max_x)*100), 
                color=color, ha='right', va="top", bbox=dict(facecolor='white', alpha=0.9,  edgecolor='none'), rotation=45)
       
        return ("{:12}".format(title), "{:6} ({:6.2f}%) /  {:6} ({:6.2f}%)".format(int(x), (x/max_x)*100,int(y), (y/max_y)*100))

    label = val_counts.name
    val_counts = pd.concat([pd.DataFrame({"ValueId":[0.99999], "#Records":[0], "#CumRecords":0}).set_index("ValueId"), val_counts], sort=False)#.reset_index(drop = True)
    
    
    if scale_val:
        val_counts = val_counts.set_index(val_counts.index/val_counts.index.max())
    max_x = val_counts.index.max()
    max_y = val_counts["#CumRecords"].max()
    
    if ax is None:
        xlim = (1 - max_x/20.0, max_x*1.05)
        
    else: 
        xlim = (min(ax.get_xlim()[0], 1 - max_x/20.0), max(ax.get_xlim()[1], max_x*1.05))
        
    
    ax = val_counts.plot(y="#CumRecords", figsize=figsize, ax=ax, color=color, label = label, drawstyle=drawstyle, xlim = xlim )
    ax.xaxis.set_major_locator(MaxNLocator(integer=True))

    stat_legend = []
    for stat in stats: 
        m_perc = re.match("^p_([0-9]+)$", str(stat))
        m_tail = re.match("^t_([0-9]+)$", str(stat))
        
        if stat == "med_y":
            median_y = val_counts[val_counts["#CumRecords"] > val_counts["#CumRecords"].max()*0.5].first_valid_index()
            stat_legend.append(add_lines(ax, val_counts, median_y, "Median Y"))

        elif stat == "med_x":
            median_x = max_x // 2 # val_counts["index".shape[0]//2
            stat_legend.append(add_lines(ax, val_counts, median_x, "Median X"))


        elif m_perc:
            val = int(m_perc.group(1))
            perc_y = val_counts[val_counts["#CumRecords"] > val_counts["#CumRecords"].max()*val/100.0].first_valid_index()
            stat_legend.append(add_lines(ax, val_counts, perc_y, "Perc "+str(val)+"% (Y)"))


        elif stat == "max":
            stat_legend.append(add_lines(ax, val_counts, max_x, "Max"))

        elif m_tail:
            val = int(m_tail.group(1))

            tail = val_counts[val_counts["#Records"] == val].first_valid_index()
            if tail:
                 stat_legend.append(add_lines(ax, val_counts, tail, str(val)+"'s"))

        elif str(stat).isdigit():
            i = int(stat)
            if i < max_x:
                stat_legend.append(add_lines(ax, val_counts, i, str(i)))
        
    return ax, pd.DataFrame(columns=["stat", label], data=stat_legend).set_index("stat")


In [ ]:
def plot_ratio(vc1, vc2, ax=None, op="div", figsize=None, color=None, xlim=None, xtics=None):

    vc_both = vc1[["#CumRecords"]].merge(vc2[["#CumRecords"]], right_index=True, left_index=True, how="outer")
    vc_both = vc_both.fillna(method='ffill')
    
    if op == "div":
        vc_both[op] =vc_both["#CumRecords_x"]/ vc_both["#CumRecords_y"]
        
    if op == "der": # kind of derivative
        
        vc_both[op] =(vc_both["#CumRecords_x"]- vc_both["#CumRecords_y"]).rolling(500).mean().diff(-50)

    elif op == "sub": 
        vc_both[op] =vc_both["#CumRecords_y"] -  vc_both["#CumRecords_x"]
    
    ax = vc_both.plot(y=op, ax=ax, figsize=figsize, label = "{} {} {}".format(vc1.name, "-" if op == "sub" else "vs ("+op+")", vc2.name) , color=color, 
                      xlim=xlim, ylim=(-50, None) if op=="der" else None)
    if xtics is not None:

        ax.set_xticks(xtics[1:-1])
    return ax


In [ ]:
def compare_val_counts(vcs, stats = [10, 100, "med_y", "med_x", "p_90","t_1", "t_2", "max"]):
    fig, axes = plt.subplots(4, 1,  figsize=(10, 10), gridspec_kw = {'height_ratios':[5, 1, 2,2 ] }, squeeze=False)
    
    colors = ["C0", "C1", "C2", "C3", "C4", "C5"]
        
    
    stat_legends = None
    for i in range(len(vcs)):
        ax, stat_legend = plot_val_count(vcs[i], ax=axes[0,0], color = colors[i % len(colors)], figsize=None, stats = stats)

        stat_legends = stat_legend if stat_legends is None else stat_legends.merge(stat_legend, left_index=True, right_index=True, how= "outer")
        

    for i, vc in enumerate(vcs[1:]):
        
        
        plot_ratio(  vcs[i], vc, op="sub", ax= axes[2,0], color=colors[i+1], xlim = axes[0,0].get_xlim(), xtics = axes[0,0].get_xticks())
        
        plot_ratio(  vcs[0], vc, op="sub", ax= axes[3,0], color=colors[i+1], xlim = axes[0,0].get_xlim(), xtics = axes[0,0].get_xticks())
        
    
    plt.tight_layout()
    axes[1, 0].axis('off')
    
    axes[0, 0].set_zorder(100)
    
    return fig, stat_legends

## Val_count_distrib

In [ ]:
def val_count_distrib(vc):
    vc = vc.groupby("#Records")["#Records"].sum()
    
    vc = vc.reindex(range(1, vc.index.max()+1)).fillna(0).astype(int)
    vc = vc.rename("nbrec").reset_index().rename(columns={"#Records": "#Values", "nbrec":"#Records"})
    vc["#DistinctValues"]= (vc["#Records"]/vc["#Values"]).astype(int)
    return vc
    

# Analysis

In [ ]:
if ".csv" in filename: 
    df = pd.read_csv(filename, usecols = col_names)
elif ".xlsx" in filename: 
    df = pd.read_excel(filename, usecols = col_names)

else: df = pd.read_table(filename)

df

In [ ]:
print("Number of occurences per tuple of values : ") 
df[col_names].reset_index().groupby(col_names).count().sort_values("index", ascending=False)#.reset_index()

In [ ]:
print("Number of occurrence per columns")
for col_name in col_names:
    print(col_name, ":")
    display(df[col_name].value_counts().reset_index())
    print("\n\n")

In [ ]:
val_counts = []
for col_name in col_names:
    val_counts.append(get_val_count(filename, col_name))

## Plots

In [ ]:
val_count_fig, val_count_tbl = compare_val_counts(val_counts, stats = ["t_1", "max"])
val_count_tbl.fillna("[NA]")

In [ ]:
val_count_fig.savefig("plot.svg")
val_count_fig.savefig("plot.png")
